# Batching editing publication dates in OJS using API

This code will use the OJS 3.3 API to batch edit the publication date for published articles ("publications"). in the archive of an OJS journal. Here's the link to the API documentation: https://docs.pkp.sfu.ca/dev/api/ojs/3.3

The journal's back issues were all uploaded at one time, long after print publication. The publication dates in OJS match the date of OJS upload, not of journal publication. This has implications for harvesting, citation output, DOI registration (and cost), and copyright.

The current OJS API can only make get requests for issues, not put. That means I had to update all the issue publication dates through the web interface.

## Workflow

Schematically:
1. GET list of all issues (filter 2016 and prior)
2. In a loop:
    1. Pull the issue IDs
    2. use issue IDs to GET issue data
    3. pull pub date
    4. pull sub and pub IDs from attached articles
    5. in a loop:
        1. use sub and pub ID to PUT unpublish publication
        2. PUT edit publication date
        3. PUT edit publish publication

## Managing credentials and endpoints

We need a list of endpoints and the API token. Problems is, the endpoints will be generated dynamically by the submission and publication IDs so I'm not sure how that will work in the implementation.

For now I'm using placeholders and I've got CSV of credentials looking like this (two rows: one header, one values):

* `token`: `SUPER TOP SECRET API TOKEN`
* `subs_endpoint`: `https://historicalpapers.journals.yorku.ca/index.php/historicalpapers/api/v1/submissions`
* `unpub_endpoint`: `https://historicalpapers.journals.yorku.ca/index.php/historicalpapers/api/v1/submissions/{submissionId}/publications/{publicationId}/unpublish`
* `edit_endpoint`: `https://historicalpapers.journals.yorku.ca/index.php/historicalpapers/api/v1/submissions/{submissionId}/publications/{publicationId}`
* `pub_endpoint`: `https://historicalpapers.journals.yorku.ca/index.php/historicalpapers/api/v1/submissions/{submissionId}/publications/{publicationId}/publish`

So let's load that up. Then, assign the token and endpoints to variables

In [1]:
# import required libraries
import os # need this to run outside of jupyter
import pandas as pd # used to manage CSV files instead of CSV library
import requests # API calls
import json # handle the output
import datetime # to handle the publication date

# set the working directory
my_dir = 'C:\\Users\\tmrozesws\\Documents\\Historical Society pub dates' # wd set here
os.chdir(my_dir)

# open the CSV files with credentials and endpoints
my_keys = pd.read_csv("CSCH_creds.csv")

# assign values using cell index location in CSV
key = my_keys.iat[0,0] # API token
issueList_endpoint = my_keys.iat[0,1] # get list of all issues
issue_endpoint = my_keys.iat[0,2] # get issue by ID
unpub_endpoint = my_keys.iat[0,3] # unpub by sub & pub ID
edit_endpoint = my_keys.iat[0,4] # edit by sub & pub ID
pub_endpoint = my_keys.iat[0,5] # repub by sub & pub ID

## Get list of all issues

EZ PZ. We've done this before.

In [2]:
# API GET call
issueList_call = requests.get(
issueList_endpoint,
params={'apiToken':key,'isPublished':'true'}
)

# assign the json output of the call to variable z
y = json.dumps(issueList_call.json())
z = json.loads(y)

# bit of code to print the output to make sure it worked the first time - keep commented out
# json_object = json.dumps(z, indent=4)
# with open("sample_fullissue.json", "w") as outfile:
#     outfile.write(json_object)

## Get issue IDs

Now we have to loop through the JSON file to pull out the issue IDs.

The structure is:
```
{
    "items": [
        {
            "id":"2264"
        },
        {
            "id":"2275"
            ...
```
We'll have to adapt this snippet of code (the next operator) from the monthly stats to pull out the IDs:  
`d = next(item for item in c if item["date"] == monthLookup)`  
In that, c is the same at z

In [16]:
z["items"]

[{'_href': 'https://historicalpapers.journals.yorku.ca/index.php/historicalpapers/api/v1/issues/2275',
  'coverImageAltText': {'en_US': ''},
  'coverImageUrl': {'en_US': ''},
  'datePublished': '2020-06-01 00:00:00',
  'description': {'en_US': ''},
  'galleys': [{'crossref::batchId': None,
    'crossref::failedMsg': None,
    'crossref::registeredDoi': None,
    'crossref::status': None,
    'datacite::registeredDoi': None,
    'datacite::status': None,
    'doaj::status': None,
    'file': None,
    'id': 6,
    'isApproved': None,
    'label': 'PDF',
    'locale': 'en_US',
    'pub-id::publisher-id': None,
    'publicationId': None,
    'seq': None,
    'submissionFileId': None,
    'urlPublished': 'https://historicalpapers.journals.yorku.ca/index.php/historicalpapers/issue/view/2275/6',
    'urlRemote': None}],
  'id': 2275,
  'identification': '2020',
  'number': None,
  'publishedUrl': 'https://historicalpapers.journals.yorku.ca/index.php/historicalpapers/issue/view/2275',
  'titl

That gives us a list of dictionaries, so that's how to pull all the issues (dictionaries) out of the original dictionary nested in a list.